# Wallpaper generator?

##Imports+Defs

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import os

#folder to store user wallpapers
if not os.path.exists('wallpapers'):
    os.makedirs('wallpapers')


##Res presets

In [8]:

RES_PRESETS = {
    "MacBook Pro 16\" (M3)": (3456, 2234),
    "MacBook Pro 14\" (M3)": (3024, 1964),
    "iPhone 15 Pro Max": (1290, 2796),
    "iPhone 15": (1179, 2556),
    "iPhone 14": (1170, 2532),
    "Square (Preview)": (800, 800)
}


##Logic

In [9]:
# Updated 2025 Device Presets (Width x Height)
RES_PRESETS = {
    "MacBook Pro 16\" (M3/M4)": (3456, 2234),
    "MacBook Pro 14\" (M3/M4)": (3024, 1964),
    "iPhone 15/16 Pro Max": (1290, 2796),
    "iPhone 15/16": (1179, 2556),
    "iPhone 14 Pro Max": (1284, 2778),
    "Standard Desktop (4K)": (3840, 2160),
    "Standard Android (FHD+)": (1080, 2400),
    "Square (Preview)": (1000, 1000)
}

def calculate_art(equation, device_res, high_res=False):
    """Generates the coordinate grid and evaluates math."""
    w, h = RES_PRESETS[device_res]

    # Scale down for fast preview; use 100% for download
    scale = 1.0 if high_res else 0.4
    res_w, res_h = int(w * scale), int(h * scale)

    x = np.linspace(-10, 10, res_w)
    y = np.linspace(-10, 10, res_h)
    X, Y = np.meshgrid(x, y)
    grid_shape = (res_h, res_w)

    # Dictionary of available functions
    safe_dict = {
        'np': np, 'sin': np.sin, 'cos': np.cos, 'tan': np.tan, 'abs': np.abs,
        'sqrt': np.sqrt, 'exp': np.exp, 'X': X, 'Y': Y,
        'normal': lambda: np.random.normal(0, 1, grid_shape),
        'uniform': lambda: np.random.uniform(0, 1, grid_shape),
        'poisson': lambda: np.random.poisson(1, grid_shape)
    }

    return eval(equation, {"__builtins__": None}, safe_dict), (w, h)


##UI

In [11]:
# UI Components
eq_input = widgets.Text(value='sin(X**2 + Y**2) + normal()*0.1', description='Formula:', layout={'width': '400px'})
cmap_input = widgets.Dropdown(options=['magma', 'inferno', 'twilight', 'ocean', 'prism', 'viridis', 'coolwarm'], value='magma', description='Theme:')
res_input = widgets.Dropdown(options=list(RES_PRESETS.keys()), value='iPhone 15/16', description='Device:')

btn_refresh = widgets.Button(description="🔄 Refresh Preview", button_style='info')
btn_download = widgets.Button(description="💾 Download Wallpaper", button_style='success')
output_area = widgets.Output()

def handle_refresh(b):
    with output_area:
        clear_output(wait=True)
        try:
            Z, (w, h) = calculate_art(eq_input.value, res_input.value, high_res=False)
            plt.figure(figsize=(6, 6 * (h/w)))
            plt.imshow(Z, cmap=cmap_input.value, origin='lower')
            plt.axis('off')
            plt.show()
        except Exception as e:
            print(f"Error in equation: {e}")

def handle_download(b):
    with output_area:
        # Fail-safe: ensure files is imported even if runtime reset
        try:
            from google.colab import files
        except ImportError:
            print("Error: Could not import google.colab files. Are you running this in Colab?")
            return

        print("Rendering high-res file... please wait.")
        try:
            Z, (w, h) = calculate_art(eq_input.value, res_input.value, high_res=True)
            # Higher DPI for crisp 2025 display standards
            fig = plt.figure(figsize=(w/300, h/300), dpi=300)
            plt.imshow(Z, cmap=cmap_input.value, origin='lower')
            plt.axis('off')

            fname = f"wallpaper_{res_input.value.replace('/', '_')}.png"
            plt.savefig(fname, bbox_inches='tight', pad_inches=0)
            plt.close(fig)

            # Browser-side download trigger
            files.download(fname)
            print("✅ Download started!")
        except Exception as e:
            print(f"Save Error: {e}")

btn_refresh.on_click(handle_refresh)
btn_download.on_click(handle_download)

# Display everything
ui_box = widgets.VBox([eq_input, cmap_input, res_input, widgets.HBox([btn_refresh, btn_download])])
display(ui_box, output_area)


Output()